<a href="https://colab.research.google.com/github/EsraaMosaad/NTI_training/blob/main/emotion_eeg_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.layers import LSTM,Dropout,Dense,TimeDistributed,Conv1D,MaxPooling1D,Flatten
from keras.models import Sequential
import tensorflow as tf
import numpy as np

In [ ]:
data=pd.read_csv('/content/emotions.csv')
# x=data[:,0:]
data.shape
x=data.iloc[:,0:2548]
data.shape
label=data.iloc[:,-1]
label
data.shape

NameError: ignored

NEGATIVE=100 and POSITIVE=010 and NEUTRAL= 001

In [ ]:
data

In [ ]:
def tran_mathscore(x):
    if x=='NEGATIVE':
      return 0
    elif x=='POSITIVE':
      return 1
    elif x=='NEUTRAL':
      return 2

In [ ]:
y=label.apply(tran_mathscore)
print(y[:10],label[:10])

0    0.0
1    2.0
2    1.0
3    1.0
4    2.0
5    2.0
6    1.0
7    0.0
8    2.0
9    0.0
Name: label, dtype: float64 0    NEGATIVE
1     NEUTRAL
2    POSITIVE
3    POSITIVE
4     NEUTRAL
5     NEUTRAL
6    POSITIVE
7    NEGATIVE
8     NEUTRAL
9    NEGATIVE
Name: label, dtype: object


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.40)
x_train.shape

(628, 2548)

In [ ]:
tf.keras.layers.GRU?

In [ ]:
inputs = tf.keras.Input(shape=(x_train.shape[1],))

expand_dims = tf.expand_dims(inputs, axis=2)
print(expand_dims.shape)

gru = tf.keras.layers.GRU(256, return_sequences=True)(expand_dims)

flatten = tf.keras.layers.Flatten()(gru)

outputs = tf.keras.layers.Dense(3, activation='softmax')(flatten)


model = tf.keras.Model(inputs=inputs, outputs=outputs)
print(model.summary())

(None, 2548, 1)
Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 2548)]            0         
                                                                 
 tf.expand_dims_4 (TFOpLambd  (None, 2548, 1)          0         
 a)                                                              
                                                                 
 gru_3 (GRU)                 (None, 2548, 256)         198912    
                                                                 
 flatten_3 (Flatten)         (None, 652288)            0         
                                                                 
 dense_3 (Dense)             (None, 3)                 1956867   
                                                                 
Total params: 2,155,779
Trainable params: 2,155,779
Non-trainable params: 0
_________________________________

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    batch_size=32,
    epochs=50,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )
    ]
)

Epoch 1/50
43/43 [==============================] - 277s 6s/step - loss: 34.4395 - accuracy: 0.7940 - val_loss: 11.6800 - val_accuracy: 0.8211
Epoch 2/50
43/43 [==============================] - 268s 6s/step - loss: 4.8529 - accuracy: 0.9098 - val_loss: 1.8596 - val_accuracy: 0.9589
Epoch 3/50
43/43 [==============================] - 279s 7s/step - loss: 1.8183 - accuracy: 0.9516 - val_loss: 1.8242 - val_accuracy: 0.9531
Epoch 4/50
43/43 [==============================] - 270s 6s/step - loss: 0.8900 - accuracy: 0.9765 - val_loss: 0.5212 - val_accuracy: 0.9707
Epoch 5/50
43/43 [==============================] - 270s 6s/step - loss: 0.2636 - accuracy: 0.9853 - val_loss: 4.0474 - val_accuracy: 0.9032
Epoch 6/50
43/43 [==============================] - 269s 6s/step - loss: 1.8843 - accuracy: 0.9648 - val_loss: 1.7552 - val_accuracy: 0.9531
Epoch 7/50
43/43 [==============================] - 268s 6s/step - loss: 0.3603 - accuracy: 0.9941 - val_loss: 1.0700 - val_accuracy: 0.9795
Epoch 8/50


In [ ]:
model.evaluate(x_test, y_test)

14/14 [==============================] - 1s 54ms/step - loss: nan - accuracy: 0.9452


[nan, 0.9452381134033203]

In [ ]:
model.predict(x_test)

array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.]], dtype=float32)